# Mastermind

Mastermind is the following game:

> *There are 7 colours to choose from. 4 colored pegs are chosen and placed 4 slots in order. This is hidden from the player.  The player places combinations of 4. As feedback the player recieves the number of correct colors and, amoung the remaining pegs, the number of correct colours in the incorrect place. The player wins when the correct 4 colours are placed in the correct order.*

The book of "The Cross-Entropy Method" by Rubenstien and Kroese discusses Mastermind  using the Cross-Entropy Method as a solution method. Here it is not obvious how to approach the problem as a dynamic program (with a reasonable state space), in this sense it is a valid alternative to a DP approach. Also it is much computationally faster than exhaustive approaches. These find a solution with far fewer guesses but do not scale well as we increaase the number of colors and slots. This highlights the advantages of the approach as a heuristic. Nonetheless there are straightforward solutions for this problem that do better than the cross entropy method. 

In [1255]:
import string
import numpy as np

import stochastic_control as sc
from mastermind import Mastermind

from IPython.display import clear_output

%load_ext autoreload
%autoreload 2

def reward(feedback):
    return 2*feedback['correct']+1*feedback['correct_color']

def consistency_check(guess,feedbacks,num_slots = 4,num_colors = 7):
    guess_works = True
    mm_guess = Mastermind(num_slots,num_colors,solution=guess,)
    for idx, feedback in enumerate(feedbacks):
        guess_feedback, done = mm_guess.step(feedback['guess'])
        if feedback != guess_feedback :
            guess_works = False
            break
    return guess_works

def generate_all_solutions(num_slots = 4,num_colors = 7, shuffle=True):
    soln = np.array([0 for _ in range(num_slots)])
    solutions = np.array([soln])

    done = False
    idx = 0
    while not done:    
        if soln[idx] >= num_colors:
            soln[idx] = 0
            idx +=1       
        else:
            solutions=np.append(solutions,[soln],axis=0)
            idx = 0   

        if idx < num_slots:
            soln[idx] += 1  
        else:
            done = True
        
    if shuffle :
        np.random.shuffle(solutions)          
            
    return solutions

## Exhaustive

Here we go through a list all solutions one-by-one, only proposing guesses that are consistent with all prior feedback. This works very well but is expensive. A related solution is the following:

> Knuth, Donald E. "The computer as master mind." Journal of Recreational Mathematics 9.1 (1976): 1-6.

In [1297]:
num_slots=4
num_colors=7
slots = range(num_slots)
colors = range(num_colors)

mm = Mastermind(num_slots,num_colors)
print(mm.solution)

[6 0 4 5]


In [1298]:
guesses = None
rewards = None
feedbacks = []

guesses = generate_all_solutions(num_slots,num_colors, shuffle=True)
for idx, guess in enumerate(guesses):
    consistent = consistency_check(guess,feedbacks,num_slots,num_colors)
    if consistent:
        feedback, done = mm.step(guess)
        feedbacks.append(feedback)
        print(feedback)
    
print('\nSolution: ',guess)
print('Number of guesses: ', len(feedbacks))    

{'guess': array([0, 4, 0, 0]), 'correct': 0, 'correct_color': 2}
{'guess': array([1, 0, 4, 4]), 'correct': 2, 'correct_color': 0}
{'guess': array([6, 0, 5, 4]), 'correct': 2, 'correct_color': 2}
{'guess': array([6, 0, 4, 5]), 'correct': 4, 'correct_color': 0}

Solution:  [5 2 1 5]
Number of guesses:  4


## Cross Entropy Method

You will notice it is not very obvious how to formulate this problem as dynamic program without having a really enormous state space (because the set of states are very big for larger numbers of slots and colors). However, you can implement the cross entropy method. This is proposed in Chapter 8 "Applications of CE to Machine Learning" from the following book:

> Rubinstein, Reuven Y., and Dirk P. Kroese. The cross-entropy method: a unified approach to combinatorial optimization, Monte-Carlo simulation and machine learning. Springer Science & Business Media, 2013.

In [1300]:
num_slots=10
num_colors=7
slots = range(num_slots)
colors = range(num_colors)

mm = Mastermind(num_slots,num_colors)
cems = [sc.CrossEntropyCategorical(np.ones(num_colors)/num_colors,
                                   rho=0.5,
                                   alpha=1.) 
                                        for slot in range(num_slots)]

print(mm.solution)

[6 1 1 3 5 6 1 6 1 0]


In [1301]:
N=100
feedbacks = []

iters = 0
done = False
while not done :
    iters+=1
    
    guesses = None
    rewards = None
    
    for _ in range(N):
        guess = np.array([cem.act() for cem in cems])   

        # evaluate the guess
        feedback, done = mm.step(guess)
        rwrd = reward(feedback)  
        if done :
            solution = guess
            break
        
        rewards = np.append(rewards,rwrd) if rewards is not None else np.array([rwrd])
        guesses = np.append(guesses,[guess],axis=0) if guesses is not None else np.array([guess])

    for slot, cem in enumerate(cems) :
        cuttoff, x = cem.train(guesses[:,slot],rewards)      
    
    clear_output()
    print('rewards:', rwrd,'\nnumber of guesses: ',iters*N)

    
print('\nSolution:',solution)


rewards: 20 
number of guesses:  400

Solution [6 1 1 3 5 6 1 6 1 0]


## Polynomial $n\times k$

It is worth noting that, despite being proposed previously, the cross-entropy method is not the best approach to this problem. The state space is $k^n$ where $k$ is the number of colors and $n$ is the number of slots. E.g. $10$ colors and $82$ slots gives more solutions than there atoms in the universe. Scarily big no...?

The following algorithm simply checks each color each slot one-by-one and returns a solution in $n\times k$ time. This solution approach can be improved further:

> V. Chvatal, Mastermind, Combinatorica 3 (1983), 325–329.

> Erdos, Paul, and Alfréd Rényi. "On two problems of information theory." Magyar Tud. Akad. Mat. Kutató Int. Közl 8 (1963): 229-243.

These solutions obviously do not generalize to general MDP/RL problems. However, it is worth noting that not all "unapproachably big" dynamic programming problems need or should be solved via function approximation RL methods.

In [1309]:
num_slots=10
num_colors=82
slots = range(num_slots)
colors = range(num_colors)
mm = Mastermind(num_slots,num_colors)
print(mm.solution)

[32 28 72 48 19 79 59 36 57  5]


In [1310]:
correct_solution = np.zeros(num_slots, dtype=int)
num_guesses = 0
for slot in slots:
    last_feedback = None
    guess = np.zeros(num_slots, dtype=int)     
    for color in colors:
        guess[slot] = color
        feedback, done = mm.step(guess)
        num_guesses += 1
        if last_feedback is not None:
            if feedback['correct'] > last_feedback['correct']:
                correct_solution[slot] = color
            elif feedback['correct'] < last_feedback['correct']:
                correct_solution[slot] = color -1
            
        last_feedback = feedback
print('number of guesses: ',num_guesses)
print('solution: ',correct_solution)

number of guesses:  820
solution:  [32 28 72 48 19 79 59 36 57  5]
